# Extracting electrical features and plotting traces using eFEL and BluePyEfe and creating a simple model

Goal:
  - Learning to use eFEL and BluePyEfe
  - Extract efeatures from voltage recordings in order to prepare for optimisation

## Turning ephys data into python object

In [ ]:
import pathlib
from pprint import pprint
import matplotlib.pyplot as plt

import bluepyefe
import bluepyefe.extract

Let's read some information about BluePyEfe 2: https://github.com/BlueBrain/BluePyEfe/tree/BPE2/bluepyefe

One of the low level class in BluePyEfe is called "Cell". It is empty at the start but will become the recipient for the recording's data associated to the cell.

In the subdirectory `ephys` are data for four cells. Let's chose one of them and instantiate a cell object for it:

In [ ]:
from bluepyefe.cell import Cell

cell_name = "C060116A6-SR-C1"
cell = Cell(name=cell_name)

To read a data file, we need to provide the metadata associated in the form of a dictionnary:

In [ ]:
files_metadata = {
    "i_file": f"../ephys/{cell_name}/X_IDrest_ch0_386.ibw",
    "v_file": f"../ephys/{cell_name}/X_IDrest_ch1_386.ibw",
    "i_unit": "A",
    "v_unit": "V",
    "t_unit": "s",
    "dt": 0.00025,
    "ljp": 14.
}

The exact fields of this dictionary depend of the metadata already present in your file. Check the reader functions present in bluepyefe/reader.py to see which entries are needed.

The units should be the ones in which the recording file is, not the units you want the output to be in. The output units will always be ms, nA, mV.

Using this metadata the cell is able to read the data file. The data is now accessible through the "recordings" attribute. "recordings" is a list that contains as many element as recordings present in the data file:

In [ ]:
cell.read_recordings(
    protocol_data=[files_metadata], 
    protocol_name="IDrest"
)

pprint(vars(cell.recordings[0]))

Let's select the first trace and plot it:

In [ ]:
recording = cell.recordings[0]
recording.plot()

Since we did not specify a starting time, finishing time, step amplitude and holding current in the metadata, we implicitly ask BluePyEfe to infer them from the current time series.
BluePyEfe does so throught two steps:
- It tries to guess the shape of the current stimulus based on the name of the protocol. In the present case, IDRest is associated to the eCode "Step" which represent a simple current step.
- It tried to find the beginning and end of the step 
To check if the eCode was interpreted correctly, we can compare above, the original current array (in blue) with an artificial one generated from the ecode_params (in orange).

If you are unhappy with the results (for example the orange line does match with the actual current), you can set the eCode parameters by hand through the metadata. For a step current, the parameters are :'ton', 'toff', 'tend', 'dt', 'amp', and 'hypamp'. For example, if we want to specify the ton and toff by hand, we would do:

    files_metadata = {
        "i_file": "../tests/exp_data/B95_Ch0_IDRest_107.ibw",
        "v_file": "../tests/exp_data/B95_Ch3_IDRest_107.ibw",
        "i_unit": "pA",
        "v_unit": "mV",
        "t_unit": "s",
        "dt": 0.00025,
        "ljp": 14.,
        "ton": 700., 
        "toff": 2700.
    }


Now that we have a voltage time series, we can get efeatures from it.

The computation of electrical features from voltage series is done by eFEL, let's see what it looks like:
https://github.com/BlueBrain/eFEL

A list of all the features available can be found in the documentation of the eFEL package https://efel.readthedocs.io/en/latest/eFeatures.html and https://bluebrain.github.io/eFEL/efeature-documentation.pdf. 

For now, let's make a list with a few features and extract them from all the recordings labelled as "IDrest" protocols:

In [ ]:
interesting_efeatures = [
    'Spikecount',
    'mean_frequency',
    'ISI_CV',
    'AP1_amp',
    'AP_width'
]

In [ ]:
cell.extract_efeatures(
    protocol_name='IDrest', 
    efeatures=interesting_efeatures
)

pprint(cell.recordings[0].efeatures)
recording.plot()

Now let's create a model that can reproduce these features !

## Let's create a simple model using BluePyOpt

Let's have a look at BluePyOpt: https://github.com/BlueBrain/BluePyOpt

First a template that will describe the cell has to be defined. A template consists of:
* a morphology
* model mechanisms
* model parameters

A morphology can be loaded from a file (SWC or ASC), here it will be a simple cyinrical morphology:

In [ ]:
import bluepyopt as bpop
import bluepyopt.ephys as ephys

In [ ]:
morph = ephys.morphologies.NrnFileMorphology('simple.swc')

By default a Neuron morphology has the following sectionlists: somatic, axonal, apical and basal. Let's create an object that points to the somatic sectionlist. This object will be used later to specify where mechanisms have to be added etc.

In [ ]:
soma_loc = ephys.locations.NrnSeclistLocation('somatic', seclist_name='somatic')

all_loc = ephys.locations.NrnSeclistLocation("all", seclist_name="all")

### Creating a mechanism

Now we can add ion channels to this morphology. Let's add a generic potassium and sodium mechanisms to the soma:

In [ ]:
mechanisms = []

for mech_name in ["SKv3_1", "NaTg"]:

    mech = ephys.mechanisms.NrnMODMechanism(
        name=f'{mech_name}.somatic',
        suffix=mech_name,
        locations=[soma_loc]
    )

    mechanisms.append(mech)

mechanisms.append(
    ephys.mechanisms.NrnMODMechanism(
        name="pas",
        suffix="pas",
        locations=[all_loc]
    )
)

The 'name' field can be chosen by the user, this name should be unique. The 'suffix' points to the same field in the NMODL file of the channel. 'locations' specifies which sections the mechanism will be added to.

### Creating parameters

Next we need to specify the parameters of the model. A parameter can be in two states: frozen and not-frozen. When a parameter is frozen it has an exact value, otherwise it only has some bounds but the exact value is not known yet.

Let's define first a parameter that sets the capacitance of the soma:

In [ ]:
cm_param = ephys.parameters.NrnSectionParameter(
    name='cm',
    param_name='cm',
    bounds=[0.5, 20.0],
    locations=[soma_loc],
    frozen=False
)

And parameters that represent the maximal conductance of the sodium and potassium channels. These two parameters will be optimised later.

In [ ]:
gk_param = ephys.parameters.NrnSectionParameter(                                    
    name='gSKv3_1bar_SKv3_1.somatic',
    param_name='gSKv3_1bar_SKv3_1',
    locations=[soma_loc],
    bounds=[0.01, 20.0],
    frozen=False,
    value_scaler=ephys.parameterscalers.NrnSegmentLinearScaler(),
)

gna_param = ephys.parameters.NrnSectionParameter(
    name='gNaTgbar_NaTg.somatic',
    param_name='gNaTgbar_NaTg',
    bounds=[0.01, 15.0],
        locations=[soma_loc],
    frozen=False,
    value_scaler=ephys.parameterscalers.NrnSegmentLinearScaler(),
)

Add parameters that represents the reversal potentials and passive current:

In [ ]:
g_pas = ephys.parameters.NrnSectionParameter(
    name='g_pas',
    param_name='g_pas',
    value=7e-04,
    locations=[soma_loc],
    frozen=True
)

e_pas = ephys.parameters.NrnSectionParameter(
    name='e_pas',
    param_name='e_pas',
    value=-62,
    locations=[soma_loc],
    frozen=True
)

ena = ephys.parameters.NrnSectionParameter(
    name='ena',
    param_name='ena',
    value=50,
    locations=[soma_loc],
    frozen=True
)

ek = ephys.parameters.NrnSectionParameter(
    name='ek',
    param_name='ek',
    value=-90,
    locations=[soma_loc],
    frozen=True
)

### Creating the template

To create the cell template, we pass all these objects to the constructor of the template

In [ ]:
cell_model = ephys.models.CellModel(
    name=f'model_{cell_name.replace("-", "_")}',
    morph=morph,
    mechs=mechanisms,
    params=[cm_param, gk_param, gna_param, g_pas, e_pas, ena, ek]
)  

Now we can print out a description of the cell

In [ ]:
print(cell_model)

With this cell we can build a cell evaluator.

## Setting up a cell evaluator

To optimise the parameters of the cell we need to create cell evaluator object. 
This object will need to know which protocols to inject, which parameters to optimise, etc.

### Creating the protocols

A protocol consists of a set of stimuli, and a set of responses (i.e. recordings). These responses will later be used to calculate the score of the parameter values.

Let's create a protocol that mimics what we had in our ephys data, that is, a square pulses at the center of the soma.

We first need to create a location object

In [ ]:
somatic_loc = ephys.locations.NrnSeclistCompLocation(
    name='soma',
    seclist_name='somatic',
    sec_index=0,
    comp_x=0.5
)

and then the stimuli, recordings and protocols. For each protocol we add a recording and a stimulus in the soma using our BluePyEfe Recording object:

In [ ]:
protocol_name = "IDrest"

holding_stimulus = ephys.stimuli.NrnSquarePulse(
    step_amplitude=recording.hypamp,
    step_delay=0,
    step_duration=recording.tend,
    location=somatic_loc,
    total_duration=recording.tend
)

step_stimulus = ephys.stimuli.NrnSquarePulse(
    step_amplitude=recording.amp,
    step_delay=recording.ton,
    step_duration=recording.toff - recording.ton,
    location=somatic_loc,
    total_duration=recording.tend
)

rec = ephys.recordings.CompRecording(
    name='%s.soma.v' % protocol_name,
    location=somatic_loc,
    variable='v'
)

protocol = ephys.protocols.SweepProtocol(protocol_name, [holding_stimulus, step_stimulus], [rec])

### Running a protocol on a cell

Now we're at a stage where we can actually run a protocol on the cell. We first need compile the mechanisms (mod files):

In [ ]:
!nrnivmodl mechanisms

and instantiate a NEURON simulator 

In [ ]:
sim = ephys.simulators.NrnSimulator()

The run() method of a protocol accepts a cell model, a set of parameter values and a simulator

In [ ]:
default_params = {"gSKv3_1bar_SKv3_1.somatic": 0.1, "gNaTgbar_NaTg.somatic": 0.1, "cm": 4.5}

responses = protocol.run(cell_model=cell_model, param_values=default_params, sim=sim)

Plotting the response traces is now easy:

In [ ]:
plt.plot(
    responses[f'{protocol_name}.soma.v']['time'],
    responses[f'{protocol_name}.soma.v']['voltage']
)

That we have a running model, we need to optimize its parameters in order for it to reproduce the data we have.

The first step to do so is to be able to measure how well or bad our model performs with respect to the ephys data.

### Adding targets to our evaluator

In BluePyOpt, the score of a model is called the `fitness`, it is computed using a fitness calculator that computes how far from the expected electrical features chosen as targets the model is.

To define a fitness calculator, we first need to create an object for each target:

In [ ]:
objectives = []
print(recording.efeatures)
for feature_name in interesting_efeatures:
    if feature_name in recording.efeatures:

        recording_names = {'': 'IDrest.soma.v'}
        
        feature = ephys.efeatures.eFELFeature(
            feature_name,
            efel_feature_name=feature_name,
            recording_names=recording_names,
            stim_start=recording.ton,
            stim_end=recording.toff,
            exp_mean=recording.efeatures[feature_name],
            exp_std=abs(0.1 * recording.efeatures[feature_name]),
            threshold=-20
        )

        objective = ephys.objectives.SingletonObjective(
            feature_name,
            feature
        )
        objectives.append(objective)

A fitness calculator can then be instantiated from these objectives and attached to an evaluator that contains both the protocol and the objectives for the protocol:

In [ ]:
fitcalc = ephys.objectivescalculators.ObjectivesCalculator(objectives)

evaluator = ephys.evaluators.CellEvaluator(
    cell_model=cell_model,
    param_names=list(default_params.keys()),
    fitness_protocols={"IDrest": protocol},
    fitness_calculator=fitcalc,
    sim=sim
)

print(evaluator)

Using this evaluator we can compute the features for our models as well as computing how well our default set of parameter is performing:

In [ ]:
efeatures = evaluator.evaluate_with_dicts(param_dict=default_params, target='values')
print("E-features values: ", efeatures)

In [ ]:
scores = evaluator.evaluate_with_dicts(param_dict=default_params, target='scores')
print("E-features scores: ", scores)

We can express the quality of the model with a single scalar, the fitness, that we will try to minimize:

In [ ]:
print("Fitness: ", sum(scores.values()))

## Tuning the parameters using grid search

The easiest way to tune the parameter is to perform grid search over the parameter space.
To do so, we can code a simple minimizer:

In [ ]:
import numpy

best_params = None
best_fitness = numpy.inf

for gna in numpy.arange(1, 15, 1):
    for gk in numpy.arange(0.1, 0.2, 0.01):
        for cm in numpy.arange(1.0, 20, 1):
            
            params = {"gSKv3_1bar_SKv3_1.somatic": gk, "gNaTgbar_NaTg.somatic": gna, "cm": cm}

            fitness = sum(evaluator.evaluate_with_dicts(param_dict=params, target='scores').values())

            if fitness <best_fitness:
                best_fitness = fitness
                best_params = params
                print("New best params ! ", params, f" (fitness: {best_fitness})")

And see the results:

In [ ]:
responses = protocol.run(cell_model=cell_model, param_values={'gSKv3_1bar_SKv3_1.somatic': 0.18999999999999995, 'gNaTgbar_NaTg.somatic': 1, 'cm': 19.0}, sim=sim)

plt.plot(
    responses[f'{protocol_name}.soma.v']['time'],
    responses[f'{protocol_name}.soma.v']['voltage']
)

But it is extremly slow !

## Tuning the parameters using an optimizer

We can do better using an optimizer, let's try here with a single objective optimizer provided by BluePyOpt.
The optimizer will run for 10 generations using 10 individuals per generation:

In [ ]:
opt = bpop.deapext.optimisationsCMA.DEAPOptimisationCMA(
    evaluator=evaluator,
    selector_name="single_objective",
    offspring_size=10
)

final_pop, halloffame, log, hist = opt.run(max_ngen=10, cp_filename='checkpoint.pkl')

In [ ]:
responses = protocol.run(
    cell_model=cell_model,
    param_values=evaluator.param_dict(halloffame[0]),
    sim=sim
)

plt.plot(
    responses[f'{protocol_name}.soma.v']['time'],
    responses[f'{protocol_name}.soma.v']['voltage']
)

print("Fitness: ", sum(halloffame[0].fitness.values))

We can visualize how the fitness improve with the generations:

In [ ]:
plt.plot(log.select("gen"), log.select("avg"), label="Average fitness")
plt.plot(log.select("gen"), log.select("min"), label="Best fitness")

plt.yscale("log")
plt.xlabel("Generation")
plt.ylabel("Fitness")